In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from skimage import io
from glob import glob
from os import listdir

In [2]:
def dfs(img,coordinates,i,j,visited):
    if i<0 or i>=img.shape[0] or j<0 or j>=img.shape[1] or img[i][j]==0 or visited[i][j]==1:
        return
        
    visited[i][j]=1;
    if i<coordinates[0]:
        coordinates[0]=i;
    if i>coordinates[1]:
        coordinates[1]=i;
    if j<coordinates[2]:
        coordinates[2]=j;
    if j>coordinates[3]:
        coordinates[3]=j;
    
    dfs(img,coordinates,i+1,j,visited);
    dfs(img,coordinates,i,j+1,visited);
    dfs(img,coordinates,i-1,j,visited);
    dfs(img,coordinates,i,j-1,visited);

In [7]:
def GetCoordinates(img_ori,img,istart,iend,jstart,jend):
    i_diff_start=1
    i_diff_end=1
    j_diff_start=1
    j_diff_end=1

    if istart==0:
        i_diff_start=0
    if iend==1080:
        i_diff_end=0
    if jstart==0:
        j_diff_start=0
    if jend==1280:
        j_diff_end=0
    visited=np.zeros((iend-istart+i_diff_start+i_diff_end,jend-jstart+j_diff_start+j_diff_end))
    img_temp=img[istart-i_diff_start:iend+i_diff_end,jstart-j_diff_start:jend+j_diff_end]
    #print(img_temp.shape)
    #print(visited.shape)
    clist=[]
    for i in range(0,50+i_diff_end+i_diff_start):
        for j in range(0,50+j_diff_end+j_diff_start):
            #print("Done with {},{}".format(i,j))
            #print(img_temp.shape)
            if img_temp[i][j]==0 or visited[i][j]==1:
                continue
            c=[i,i,j,j]
            dfs(img_temp,c,i,j,visited)
            #print("Top:{} Bottom:{} Left:{} Right:{}".format(c[0],c[1],c[2],c[3]))
            c[0]=c[0]+istart-i_diff_start
            c[1]=c[1]+istart-i_diff_start
            c[2]=c[2]+jstart-j_diff_start
            c[3]=c[3]+jstart-j_diff_start
            
            ## If top = bottom, left = right, and is inside our predefined boundary
            if c[0]!=c[1] and c[2]!=c[3] and c[0]>=istart and c[1]<iend and c[2]>=jstart and c[3]<jend:
                #print("To Add: Top:{} Bottom:{} Left:{} Right:{}".format(c[0],c[1],c[2],c[3]))
                if not (img_ori[c[0]:c[1]+1,c[2]:c[3]+1,0]==0).all() and  not (img_ori[c[0]:c[1]+1,c[2]:c[3]+1,1]==0).all() and not (img_ori[c[0]:c[1]+1,c[2]:c[3]+1,0]==2).all():
                    clist.append(c)
    
    if len(clist)>0:
        clist=np.array(clist)
        df=pd.DataFrame({'top':clist[:,0],'bottom':clist[:,1],'left':clist[:,2],'right':clist[:,3]})
        return df
    else:
        return None

In [4]:
folders_composite=glob("Composite Images/*/")
folders_composite

['Composite Images\\20190508-C2GNT1-EGFP-Noc-01-Data Preparation\\',
 'Composite Images\\20190508-C2GNT1-EGFP-Noc-02-Data Preparation\\',
 'Composite Images\\20190508-C2GNT1-EGFP-Noc-03-Data Preparation\\',
 'Composite Images\\20190508-C2GNT1-EGFP-Noc-04-Data Preparation\\',
 'Composite Images\\20190508-C2GNT1-EGFP-Noc-05-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-01-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-02-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-03-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-04-Data Preparation\\',
 'Composite Images\\20190508-GALNT4-EGFP-Noc-05-Data Preparation\\',
 'Composite Images\\20190508-GALNT8-EGFP-Noc-01-Data Preparation\\',
 'Composite Images\\20190508-GALNT8-EGFP-Noc-02-Data Preparation\\',
 'Composite Images\\20190508-GALNT8-EGFP-Noc-03-Data Preparation\\',
 'Composite Images\\20190508-GALNT8-EGFP-Noc-04-Data Preparation\\',
 'Composite Images\\20190508-GALNT

In [5]:
labels=pd.read_csv('labels.csv',index_col=0)
labels

,label
0,20190508-C2GNT1-EGFP-Noc-01
1,20190508-C2GNT1-EGFP-Noc-02
2,20190508-C2GNT1-EGFP-Noc-03
3,20190508-C2GNT1-EGFP-Noc-04
4,20190508-C2GNT1-EGFP-Noc-05
5,20190508-GALNT4-EGFP-Noc-01
6,20190508-GALNT4-EGFP-Noc-02
7,20190508-GALNT4-EGFP-Noc-03
8,20190508-GALNT4-EGFP-Noc-04
9,20190508-GALNT4-EGFP-Noc-05


In [6]:
sys.setrecursionlimit(1000000)

In [11]:
for k in range(0,40):

    img_file_path=folders_composite[k]+"composite_{}.tif".format(labels.iloc[k].values[0])
    img=io.imread(img_file_path)
    
    img_non_golgy_file_path=folders_composite[k]+"non_golgy_object.tif"
    img_non_golgy_obj=io.imread(img_non_golgy_file_path)

    non_golgi_df=pd.DataFrame(columns=['top','bottom','left','right'])
    
    ## Extract coordinates for non-golgy objects
    for i in range(0,1031,2):
        print("Processing {} - {} now: row {} to row {}".format(k,labels.iloc[k].values[0],i,i+50))
        for j in range(0,1231,2):
            if (img_non_golgy_obj[i:i+50,j:j+50]==0).sum()<=50 or (img_non_golgy_obj[i:i+50,j:j+50]==1).sum()<=30:
                continue
            cur_co_df=GetCoordinates(img,img_non_golgy_obj,i,i+50,j,j+50)
            if cur_co_df is not None:
                non_golgi_df=non_golgi_df.append(cur_co_df).reset_index(drop=True)


    non_golgy_obj_co_file_path=folders_composite[k]+"\\{}_non_golgy_coordinates.csv".format(labels.iloc[k].values[0])
    non_golgi_df.to_csv(non_golgy_obj_co_file_path)

    print('Done with {}'.format(labels.iloc[k].values[0]))

Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 0 to row 50
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 2 to row 52
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 4 to row 54
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 6 to row 56
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 8 to row 58
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 10 to row 60
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 12 to row 62
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 14 to row 64
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 16 to row 66
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 18 to row 68
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 20 to row 70
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 22 to row 72
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 24 to row 74
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 26 to row 76
Processing 0 - 20190508-C2GNT1-EGFP-Noc-01 now: row 28 to row 78
Processing 0 - 20190508-C2GNT1